In [1]:
pip install requests pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd

In [19]:
# Testing the API 
r = requests.get('https://archive-api.open-meteo.com/v1/archive?latitude=39.0428&longitude=-89.9518&start_date=2000-01-01&end_date=2026-01-01&daily=temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,snowfall_sum,wind_speed_10m_max&timezone=America%2FChicago&temperature_unit=fahrenheit&wind_speed_unit=mph&precipitation_unit=inch')
data = r.json()

In [ ]:
# Accessing first record of max temp (for testing purposes)
day1_temp = (data['daily']['temperature_2m_max'][0])
day1_temp

54.7

In [ ]:
# Creating a dataframe from the JSON file
df = pd.DataFrame(bh_daily)
df

,time,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,snowfall_sum,wind_speed_10m_max
0,2000-01-01,54.7,30.5,43.1,0.000,0.0,14.3
1,2000-01-02,65.6,41.3,54.3,0.016,0.0,19.2
2,2000-01-03,47.5,36.0,39.6,0.854,0.0,11.6
3,2000-01-04,35.7,23.6,28.4,0.000,0.0,14.3
4,2000-01-05,34.4,21.7,27.2,0.000,0.0,12.0
...,...,...,...,...,...,...,...
9493,2025-12-28,72.0,23.5,54.7,0.071,0.0,22.8
9494,2025-12-29,24.5,14.8,19.8,0.000,0.0,23.1
9495,2025-12-30,32.3,15.2,24.2,0.000,0.0,11.4
9496,2025-12-31,39.6,28.6,32.8,0.000,0.0,13.7


In [ ]:
# Identifying data types of the DF
# Dates came in as 'time' column, and data type was 'string' which was less than ideal.
df.dtypes

date                   datetime64[us]
temperature_2m_max            float64
temperature_2m_min            float64
temperature_2m_mean           float64
precipitation_sum             float64
snowfall_sum                  float64
wind_speed_10m_max            float64
year                            int32
month                           int32
day                             int32
season                            str
dtype: object

In [ ]:
# Converting 'string' date field ('time') into datetime. 
df['time'] = pd.to_datetime(df['time'], format="%Y-%m-%d")
df = df.rename(columns= {'time':'date'}) #Renamed 'time' to 'date'
df.dtypes

date                   datetime64[us]
temperature_2m_max            float64
temperature_2m_min            float64
temperature_2m_mean           float64
precipitation_sum             float64
snowfall_sum                  float64
wind_speed_10m_max            float64
dtype: object

In [ ]:
# Splitting the date column into 'year', 'month', 'day' columns for better querying
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df

,date,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,snowfall_sum,wind_speed_10m_max,year,month,day
0,2000-01-01,54.7,30.5,43.1,0.000,0.0,14.3,2000,1,1
1,2000-01-02,65.6,41.3,54.3,0.016,0.0,19.2,2000,1,2
2,2000-01-03,47.5,36.0,39.6,0.854,0.0,11.6,2000,1,3
3,2000-01-04,35.7,23.6,28.4,0.000,0.0,14.3,2000,1,4
4,2000-01-05,34.4,21.7,27.2,0.000,0.0,12.0,2000,1,5
...,...,...,...,...,...,...,...,...,...,...
9493,2025-12-28,72.0,23.5,54.7,0.071,0.0,22.8,2025,12,28
9494,2025-12-29,24.5,14.8,19.8,0.000,0.0,23.1,2025,12,29
9495,2025-12-30,32.3,15.2,24.2,0.000,0.0,11.4,2025,12,30
9496,2025-12-31,39.6,28.6,32.8,0.000,0.0,13.7,2025,12,31


In [ ]:
# Created 'season' column and apply each month to a season. Using 'meteorological' seasons for even cutoffs rather than solstice or equinoxes.
# Dictionary that contains the meteorolgical 'season' that corresponds to each numerical month. 3 months for every season.
season_dict = {1:"Winter", 2:"Winter", 3:"Spring", 4:"Spring", 5:"Spring", 6:"Summer", 7:"Summer", 8:"Summer", 9:"Fall", 10:"Fall", 11:"Fall", 12:"Winter"}

df['season'] = df['month'].map(season_dict) # Created 'season' column and ran map function to fill in season based on int value in 'month' column
df[df['month'] == 7].head() # Used to verify records were populating season correctly.

,date,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,snowfall_sum,wind_speed_10m_max,year,month,day,season
182,2000-07-01,81.0,63.8,72.7,0.024,0.0,9.0,2000,7,1,Summer
183,2000-07-02,86.9,70.0,77.9,0.071,0.0,10.7,2000,7,2,Summer
184,2000-07-03,85.3,74.1,77.7,0.307,0.0,10.2,2000,7,3,Summer
185,2000-07-04,86.7,73.2,79.5,0.169,0.0,6.4,2000,7,4,Summer
186,2000-07-05,89.2,75.7,82.1,0.012,0.0,6.8,2000,7,5,Summer


In [ ]:
#Export data to csv. Did not include index since data is broken out by month/day/year/season.
df.to_csv(r"C:\Users\DJ\Documents\projects\weather_trends\data\bh_2000_2026_wx.csv", index=False)